# Evaluate Classification model

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


LOAD_DATA = load_breast_cancer()

In [2]:
# Prepare dataset.
df = pd.DataFrame(
    LOAD_DATA.data,
    columns=LOAD_DATA.feature_names
)
df['y'] = LOAD_DATA.target

# Split the data into Explanatory and Objective variables.
X = df[
    ['mean radius', 'mean texture']
]
y = df['y']

# Split dataset into training and test data.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0
)

print(X_test)

     mean radius  mean texture
512       13.400         20.52
457       13.210         25.25
439       14.020         15.66
298       14.260         18.17
37        13.030         18.42
..           ...           ...
7         13.710         20.83
408       17.990         20.66
523       13.710         18.68
361       13.300         21.57
553        9.333         21.94

[171 rows x 2 columns]


In [3]:
# Construct Random Forest classification model.
rf_classifier = RandomForestClassifier(
    max_depth=3,
    random_state=0
).fit(
    X_train,
    y_train
)

# Calculate predicted values.
y_train_prediction = rf_classifier.predict(X_train)
y_test_prediction = rf_classifier.predict(X_test)